In [3]:
from bs4 import BeautifulSoup
import requests
import json
import csv
import datetime
import time


insight_news_info = []
cnt = 0 # 뉴스 number

# csv변환시키는 함수
def convert_to_csv(csv_name, fieldnames, infor):
    try:
        with open(f'./${csv_name}.csv', 'w', newline='') as c:
            fieldnames = fieldnames
            writer = csv.DictWriter(c, fieldnames=fieldnames) 
            writer.writeheader()

            for news in infor:
                print(news)
                writer.writerow(news) 
    except:
        pass

    
# Dict형태로 변환 함수
def dict_infor(**kwargs):
    news_info = []
    context = {}
    for k,v in kwargs.items():
        context[k] = v
    news_info.append(context)
#     print(news_info)
    return news_info



for num in range(1,2):
    time.sleep(5)
    response = requests.get(f'https://www.insight.co.kr/section/money?page={num}')
    soup = BeautifulSoup(response.text, 'html.parser')

    # 메인 페이지 기사 ul
    ul = soup.select_one('body > div.content > div > div.section-wrap > div.section-list > ul')
    # 메인 페이지 기사리스트들 li
    lis = ul.select('li')
    # 현재 카테고리 네임
    category = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > span')
    press = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > a').text
    # 기사 리스트 for문
    # 프리뷰 기사 img, title, date가져옴
    # 현재 카테고리 각각의 리스트에 함께 나오게 하기 위해 단순히 위에서 받은 new_category밑에다 써줌
    
    for li in lis:
        cnt += 1
        preview_img = li.select_one('li > div > a.section-list-article-image')
        preview_img = preview_img['style'].split("(")[1].split(')')[0]
        title = li.select_one('li > div > a.section-list-article-title').text.strip()
        news_category = category.text.split('>')[1].strip()
        date = li.select_one('li > div > span.section-list-article-byline')
        link = li.select_one('li > div > a.section-list-article-title')['href']
        code = link.split('/')[4]
        
        # code로 내용을 가지고 오는대신 다시한번 requests와 parser이용
        news_url = requests.get(link)
        news_url_html = BeautifulSoup(news_url.text, 'html.parser')
        
        # 세부 컨텐츠 안에 주요 내용 selector css         
        content = news_url_html.select_one('body > div.content > div > div.news-container > div.news-wrap > div.news-article > div.news-article-memo')
        # 세부 컨텐츠 안에 이미지, 이미지 여러개이므로 list에 담아줌
        news_image = content.select('img')
        image_url_list = []
        try:
            for image in news_image:
                image_url_list.append(image['src'])

            # 기자 이름이 날짜와 함께 나오는 경우가 있음 \".\" 이 있는 경우는 기자 네임이 있는 경우라 .을 기준으로 나누어서 날짜만 가져옴
            if '·' in date.text:
                date = date.text.strip().split('·')[1]

            else:
                date = date.text.strip()
            date_list = date.replace(' ','-').split('-')
            date_list = ' '.join(date_list).split()
            date_list = date_list[0]+'-'+date_list[1]+'-'+date_list[2]+' '+date_list[3]
            print(date_list)
            date_code = datetime.datetime.strptime(date_list,'%Y-%m-%d %H:%M:%S')
    #         print(datetime.datetime.strptime(date_list,'%Y%m%d%H%M%S'))
            print(type(date_code))



            # 이하 똑같은 형식
            insight_news_info = dict_infor(num=cnt, press=press,news_code=code, news_category=news_category, date=date, preview_img=preview_img, title=title, content=content.text,img=image_url_list,ref=link, date_code= date_list)
        except:
            pass
# print(insight_news_info)
convert_to_csv('insight_news_infor',['num', 'press', 'news_code', 'news_category', 'date','preview_img','title', 'content', 'img', 'ref' ],insight_news_info)


2021-07-13 07:13:22
<class 'datetime.datetime'>
2021-06-29 16:24:29
<class 'datetime.datetime'>
2021-06-02 17:53:23
<class 'datetime.datetime'>
2021-05-24 13:07:54
<class 'datetime.datetime'>
2021-05-20 17:20:09
<class 'datetime.datetime'>
2021-05-10 14:33:15
<class 'datetime.datetime'>
2021-05-06 11:09:53
<class 'datetime.datetime'>
2021-05-04 22:06:07
<class 'datetime.datetime'>
2021-04-06 09:59:15
<class 'datetime.datetime'>
2021-02-19 16:09:14
<class 'datetime.datetime'>
2021-02-19 09:10:42
<class 'datetime.datetime'>
2021-01-20 17:57:46
<class 'datetime.datetime'>
2021-01-14 12:32:41
<class 'datetime.datetime'>
2021-01-07 11:38:12
<class 'datetime.datetime'>
2021-01-05 17:43:24
<class 'datetime.datetime'>
{'num': 15, 'press': '인사이트', 'news_code': '319555', 'news_category': '생활경제', 'date': ' 2021-01-05 17:43:24', 'preview_img': 'https://img.insight.co.kr/static/2021/01/05/300/az6m088b5m48150a3iyx.jpg', 'title': "오늘(5일)부터 '신용카드 포인트' 한 곳에 모아 계좌로 이체할 수 있다", 'content': "\n기사와 관련 없는 자료 

### 다음 뉴스 기사 스크래핑 조건

0. 해당기사가 json인지 text로 가져올 수 있는지 확인한다 -[0] text/html이다

1. 기사의 카테고리별로 스크래핑을 해온다(정치, 경제, 사회, 국제, 연예(nope), 스포츠(nope), it, 문화). 이때 해당 카테고리가 존재하지 않으면 안해도 된다. 각 카테고리의 전체기사를 스크래핑 한다

2. 날짜는 오늘 날짜로부터 30일이 지난 기사까지만 보여준다(한달전 뉴스까지 화면에 보여준다) . 스크래핑을 항상 현재 날짜부터 10일전까지의 기사를 스크래핑한다.(누적되서 30일전 기사들도 쌓일 예정이다. 이 중 우리는 30일전까지의 기사까지만 보여준다) 


3. 크론탭(celery 이용할 수 있는지 찾아보기)을 이용해 스크래핑을 1시간에 한번씩 해준다 (상황을 보고 판단할 예정)

4. 스크래핑이 도중에 에러가 날 경우에 대비한 예외처리를 해준다.

5. 기사의 세부내용(이미지, 컨텐츠) 자체를 모델에 집어넣은 후 꺼내어 화면에 뿌려줄때 이미지 형식은 이미지 태그를 써서 src에 넣어주는 로직을 짠다

6. 아티클에 넣어야 할 데이터는 
category, press(언론사), potal(포털-다음), code, preview_img, title, date, content(내용이미지, 세부내용), ref(기사 원본 링크) 

#### 1. 스크래핑 순서
https://news.daum.net/breakingnews/society?page=1&regDate=20210715  
society : 사회
politics : 정치
economic : 경제
foreign : 국제
culture : 문화
digital : it
popular : 연예
sports: 스포츠 

사회 -> 정치 -> 경제 -> 국제 -> 문화 -> IT -> 연예 -> 스포츠     

#### 2. 스크래핑 개수와 시간
> 한번 스크래핑을 할때 각 카테고리별로 50개씩 중복되지 않게 스크래핑 한다(날짜기준으로 정렬해서 모델에 넣기)
> 하루에 30분에 한번씩 스크래핑 될 수 있게 한다
#### 3. 예외처리 찾아보기

#### 4. 고려할 점
1. 생각해보면 굳이 며칠전것까지 스크래핑 할 필요는 없는듯 하다. 매일매일 스크래핑 할 텐데 굳이 옛날꺼까지 하려는 이유는 단순히 초반에는 뉴스 데이터가 없을 것이기 때문에 그러는 것 같다

In [65]:
from bs4 import BeautifulSoup
import requests
import json
import csv
from datetime import datetime
import time
import re

category_list = ['society', 'politics', 'economic', 'foreign', 'culture', 'digital', 'popular', 'sports']

for category in category_list:
    for num in range(1,4):
        time.sleep(5)
        print(f'{category} - {num} 페이지 스크래핑 시작 -!')
        date = datetime.today().strftime("%Y%m%d")
        
        response = requests.get(f'https://news.daum.net/breakingnews/{category}?page={num}&regDate={date}')
        soup = BeautifulSoup(response.text, 'html.parser')
        ul = soup.select_one('#mArticle > div.box_etc > ul')
        lis = ul.select('ul > li')
        try:
            for li in lis:
                title = li.select_one('div > strong > a').text
                ref = li.select_one('div > strong > a')['href']
                press = li.select_one('strong > span').text
                press = press.split('·')
                press = press[0]
                preview_img = li.select_one('a > img')['src']
                news_url = requests.get(ref)
                news_url_html = BeautifulSoup(news_url.text, 'html.parser')
                
                news_url_html.findAll('img',{'src':re.compile('https')})
                
                
                content = news_url_html.select_one('#harmonyContainer > section')
                print(content)
#                 pattern = r'src'
#                 matchDb = re.search(pattern, content)
#                 print(matchDb)
                a = re.findall('src="([^"]+)"',content)
                print(a)
                detail_date = news_url_html.select_one('#cSub > div > span > span:nth-child(2) > span')

#                 print(f'title: {title} , link: {ref} , press: {press} , preview: {preview_img} , detail_date: {detail_date}\n')
        except TypeError:
            print('에러발생함')
            pass

society - 1 페이지 스크래핑 시작 -!
<section dmcf-sid="kKcZ4div3r">
<figure class="figure_frm origin_fig" dmcf-pid="k00UfZHeMf" dmcf-ptype="figure">
<p class="link_figure"><img alt="전해철 행정안전부 장관이 15일 코로나19  특별방역 점검을 위해 경기 용인시 수지구 예방접종센터를 방문해 현장을 둘러보고 있다. [이미지출처=연합뉴스]" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/15/akn/20210715225333508hccf.jpg" data-org-width="550" dmcf-mid="k6DWQlmNnL" dmcf-mtype="image" height="auto" src="https://img2.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/15/akn/20210715225333508hccf.jpg" width="658"/></p>
<figcaption class="txt_caption default_figure">
            전해철 행정안전부 장관이 15일 코로나19 특별방역 점검을 위해 경기 용인시 수지구 예방접종센터를 방문해 현장을 둘러보고 있다. [이미지출처=연합뉴스]
           </figcaption>
</figure>
<p dmcf-pid="kFao2rxsoe" dmcf-ptype="general">[아시아경제 임철영 기자] 전해철 행정안전부 장관이 15일 용인시 수지구 예방접종센터와 임시선별검사소를 방문해 코로나19 대응현황 및 폭염대책 추진상황 등을 현장점검 했다.</p>
<p dmcf-pid="k1PbNcr1Dw" dmcf-ptype="general">이번 점검은 최근 수도권에서 코로나19 비상상황이 지속되는 가운데 인구밀집도가

KeyboardInterrupt: 

In [135]:
from bs4 import BeautifulSoup
import requests
import json
import csv
from datetime import datetime
import time
import re

category_list = ['society', 'politics', 'economic', 'foreign', 'culture', 'digital', 'popular', 'sports']

for num in range(1,3):
    time.sleep(5)
    print(f'{category} - {num} 페이지 스크래핑 시작 -!')
    date = datetime.today().strftime("%Y%m%d")

    response = requests.get(f'https://news.daum.net/breakingnews/society?page={num}&regDate={date}')
    soup = BeautifulSoup(response.text, 'html.parser')
    ul = soup.select_one('#mArticle > div.box_etc > ul')
    lis = ul.select('ul > li')
    try:
        for li in lis:
            title = li.select_one('div > strong > a').text
            ref = li.select_one('div > strong > a')['href']
            press = li.select_one('strong > span').text
            press = press.split('·')
            press = press[0]
            preview_img = li.select_one('a > img')['src']
            news_url = requests.get(ref)
            news_url_html = BeautifulSoup(news_url.text, 'html.parser')
            

            content = news_url_html.select_one('#harmonyContainer > section')
            lists = content.find_all(["img","p"])
            print(lists)
            



    except TypeError:
        print('에러발생함')
        pass

society - 1 페이지 스크래핑 시작 -!
에러발생함
society - 2 페이지 스크래핑 시작 -!
[<p class="link_figure"><img alt="[대전=뉴시스] 자발적으로 형성된 액시톤 신호. 중앙의 동그란 부분이 액시톤 입자에서 방출되는 광전자 신호며 가로축은 광전자의 에너지, 세로축은 광전자의 운동량이다. *재판매 및 DB 금지" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/16/newsis/20210716000108286hekf.jpg" data-org-width="388" dmcf-mid="kuWcIjrMnU" dmcf-mtype="image" height="auto" src="https://img1.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/16/newsis/20210716000108286hekf.jpg" width="658"/></p>, <img alt="[대전=뉴시스] 자발적으로 형성된 액시톤 신호. 중앙의 동그란 부분이 액시톤 입자에서 방출되는 광전자 신호며 가로축은 광전자의 에너지, 세로축은 광전자의 운동량이다. *재판매 및 DB 금지" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/16/newsis/20210716000108286hekf.jpg" data-org-width="388" dmcf-mid="kuWcIjrMnU" dmcf-mtype="image" height="auto" src="https://img1.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/16/newsis/20210716000108286hekf.jpg" width="658"/>, <p dmcf-pid="knaVH0FtIi"

[<p class="link_figure"><img alt="박범계 법무부 장관이 15일 오전 경기도 정부과천청사 법무부로 출근하며 취재진의 질문에 답변하고 있다. /뉴시스" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/16/THEFACT/20210716000027297llla.jpg" data-org-width="640" dmcf-mid="knt2ICWzMt" dmcf-mtype="image" height="auto" src="https://img2.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/16/THEFACT/20210716000027297llla.jpg" width="658"/></p>, <img alt="박범계 법무부 장관이 15일 오전 경기도 정부과천청사 법무부로 출근하며 취재진의 질문에 답변하고 있다. /뉴시스" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/16/THEFACT/20210716000027297llla.jpg" data-org-width="640" dmcf-mid="knt2ICWzMt" dmcf-mtype="image" height="auto" src="https://img2.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/16/THEFACT/20210716000027297llla.jpg" width="658"/>, <p dmcf-pid="kZZb8xlobM" dmcf-ptype="general">[더팩트ㅣ박나영 기자] 한명숙 전 총리 사건 처리 과정에 부적절한 수사관행을 확인하고 개선안을 내놓은 법무부·대검찰청 합동감찰 결과 평가가 양쪽으로 갈리고 있다. 검찰 '흠집내기'라는 비판과 부적절한 수사관행을 끊는

In [ ]:
            content = news_url_html.select_one('#harmonyContainer > section')
#             content = f'"""{content}"""'
            
#             rx = re.compile(r'<img[^>]*src="([^"]+)')
            
#             r = rx.findall(content)
#             print(r)


In [21]:
from datetime import datetime, timedelta


print(datetime.today())
print(datetime.today().year)
print(datetime.today().month)
print(datetime.today().day)
print(datetime.today().strftime("%Y%m%d"))

# 어제 날짜 가져오기
for num in range(1,10):
    yesterday = datetime.today() - timedelta(num)
    print(yesterday.strftime("%Y%m%d"))


2021-07-15 21:37:40.041790
2021
7
15
20210715
20210714
20210713
20210712
20210711
20210710
20210709
20210708
20210707
20210706


In [29]:
from datetime import datetime

date = datetime.today().strftime("%Y%m%d")
print(date)

20210715


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")
django.setup()
from scrapping.models import Press, Category, Article
import datetime

insight_news_info = []
    
# Dict형태로 변환 함수
def dict_infor(**kwargs):
    context = {}
    for k,v in kwargs.items():
        context[k] = v
    
    return context

def parse_insight():
    cnt = 0 # 뉴스 number
    data = {}
    i = 0

    try:
        for num in range(1,20):
            response = requests.get(f'https://www.insight.co.kr/section/health?page={num}')
            soup = BeautifulSoup(response.text, 'html.parser')

            ul = soup.select_one('body > div.content > div > div.section-wrap > div.section-list > ul')
            lis = ul.select('li')
            category = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > span')
            press = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > a').text
        
            for li in lis:
                cnt += 1
                preview_img = li.select_one('li > div > a.section-list-article-image')
                preview_img = preview_img['style'].split("(")[1].split(')')[0]
                print(preview_img)
                title = li.select_one('li > div > a.section-list-article-title').text.strip()
                news_category = category.text.split('>')[1].strip()
                date = li.select_one('li > div > span.section-list-article-byline')
                link = li.select_one('li > div > a.section-list-article-title')['href']
                code = link.split('/')[4]

                news_url = requests.get(link)
                news_url_html = BeautifulSoup(news_url.text, 'html.parser')
                
                content = news_url_html.select_one('body > div.content > div > div.news-container > div.news-wrap > div.news-article > div.news-article-memo')
                news_image = content.select('img')
                image_url_list = []
                
                for image in news_image:
                    image_url_list.append(image['src'])
                
                if '·' in date.text:
                    date = date.text.strip().split('·')[1]

                else:
                    date = date.text.strip()
                date_code = date.replace(' ','-').split('-')
                date_code = ' '.join(date_code).split()
                date_code = date_code[0]+'-'+date_code[1]+'-'+date_code[2]+' '+date_code[3]
                print(date_code)
                # date_code = datetime.datetime.strptime(date_code,'%Y%m%d%H%M%S')
    #         print(datetime.datetime.strptime(date_list,'%Y%m%d%H%M%S'))
                insight_news_info = dict_infor(press=press,news_code=code, news_category=news_category, date=date, date_code=date_code, preview_img=preview_img, title=title, content=content.text,img=image_url_list,ref=link )
                print(insight_news_info)            
                i += 1
                data[i] = insight_news_info
        return data
    except:
        pass
        # except:
        #     print('문제가 있네용')
        #     pass

if __name__=='__main__':

    news_dict = parse_insight()

    try:
        for v in news_dict.values():
            if not Article.objects.filter(title = v['title']):
                Article.objects.create(
                    press=Press.objects.filter(name=v['press']).first(),
                    category=Category.objects.filter(name=v['news_category']).first(),
                    code=v['news_code'],
                    date=v['date'],
                    date_code=v['date_code'],
                    preview_img=v['preview_img'],
                    title=v['title'],
                    img=v['img'],
                    content=v['content'],
                    ref=v['ref']
                    )
    except:
        pass


In [136]:
import re
junk = """<section dmcf-sid="ky3kqcsW2W">
<div class="video_frm" dmcf-pid="kfngmkQXNI" dmcf-ptype="kakaotv">
<div class="layer_vod">
<div class="vod_player">
<iframe allowfullscreen="" class="player_iframe" dmcf-mid="kdQpeH2lQF" dmcf-mtype="video/kakaotv/owner" dmcf-poster-mid="kWxvuXoz1G" frameborder="0" height="370" id="video@kdQpeH2lQF" poster="https://t1.daumcdn.net/news/202107/15/kbs/20210715234928417akwe.jpg" scrolling="no" src="//tv.kakao.com/embed/player/cliplink/420772192?service=daum_news&amp;m_use_inline=true&amp;ios_allow_inline=true&amp;m_prevent_sdk_use=true&amp;wmode=opaque" width="100%"></iframe>
</div>
</div>
</div>
<p dmcf-pid="kPbtWXhsBA" dmcf-ptype="general">[KBS 강릉] [앵커]</p>
<p dmcf-pid="knCQDyHqlS" dmcf-ptype="general">달라진 도로교통법에 따라 전동킥보드를 탈 땐 안전모를 꼭 착용해야 하고 운전면허도 있어야 합니다.</p>
<p dmcf-pid="kRDHUyRwSm" dmcf-ptype="general">그런데 면허도 없이 전동킥보드를 쉽게 빌릴 수 있다고 하는데요.</p>
<p dmcf-pid="knrJhXY2dB" dmcf-ptype="general">무엇이 문제일까요?</p>
<p dmcf-pid="kIR0XB5Pdu" dmcf-ptype="general">이청초 기자가 취재했습니다.</p>
<p dmcf-pid="kEmwlzhpy3" dmcf-ptype="general">[리포트]</p>
<p dmcf-pid="kM7xIBCz1B" dmcf-ptype="general">인도와 차도를 오가고, 역주행까지 하는 전동킥보드.</p>
<p dmcf-pid="kJWlxpHXHs" dmcf-ptype="general">안전모가 아예 없거나 있어도 들고 다니는 사람까지 가지가집니다.</p>
<p dmcf-pid="kq41EXCBIN" dmcf-ptype="general">모두 도로교통법 위반입니다.</p>
<p dmcf-pid="kYPZiE3yIG" dmcf-ptype="general">면허가 있는지 확인해 봤습니다.</p>
<p dmcf-pid="kNghnCuph4" dmcf-ptype="general">[전동킥보드 이용자 A : "(킥보드 타려면 면허 있어야 되는데, 면허 있으세요 혹시?) 아니요."]</p>
<p dmcf-pid="kItuy4gEt5" dmcf-ptype="general">교복을 입은 학생도 눈에 띕니다.</p>
<p dmcf-pid="kycoVHoBlw" dmcf-ptype="general">[전동킥보드 이용자 B : "(이거 등록할 때 면허증을 입력해야 되는 걸로 알고 있거든요?") 아 이거는 등록 안 해도 돼요."]</p>
<p dmcf-pid="kuIifmDbFE" dmcf-ptype="general">어떻게 이런 일이 가능할까?</p>
<p dmcf-pid="kCalMUE6qj" dmcf-ptype="general">공유 전동 킥보드가 많이 모여있는 장소 중 한 곳입니다. </p>
<p dmcf-pid="kDGp2f4LYo" dmcf-ptype="general">어떤 과정을 통해서 면허 없이도 이 킥보드를 이용할 수 있는지 직접 확인해보겠습니다.</p>
<p dmcf-pid="kqSxnlI8yt" dmcf-ptype="general">스마트폰으로 한 업체에 대여 신청했습니다.</p>
<p dmcf-pid="kMpqct84Zz" dmcf-ptype="general">면허 보유를 묻는 화면에, '네'라고 표시하자, 바로 킥보드를 빌릴 수 있었습니다.</p>
<p dmcf-pid="k1LZNJelwq" dmcf-ptype="general">이번엔 촬영 인증 방식을 사용하는 업체의 앱입니다.</p>
<p dmcf-pid="kf6KKhcEVK" dmcf-ptype="general">빈 보도블록을 찍자.</p>
<p dmcf-pid="kzBwZ83VBI" dmcf-ptype="general">'찰칵'</p>
<p dmcf-pid="kBaXgIwIui" dmcf-ptype="general">면허가 등록됐다는 문구가 뜹니다.</p>
<p dmcf-pid="kVlNbXv1MJ" dmcf-ptype="general">계단도.</p>
<p dmcf-pid="kfcDFuF9WB" dmcf-ptype="general">'찰칵'</p>
<p dmcf-pid="keh77vRnPB" dmcf-ptype="general">조각상도.</p>
<p dmcf-pid="kiW5Y4Q7Tz" dmcf-ptype="general">'찰칵'</p>
<p dmcf-pid="kdxltqgpHo" dmcf-ptype="general">결과는 다 마찬가집니다.</p>
<p dmcf-pid="kLvGkbRnIU" dmcf-ptype="general">KBS가 대여업체 6곳의 앱을 시험해 봤더니 4곳의 인증과정이 허술했습니다.</p>
<p dmcf-pid="kkALb3RhyN" dmcf-ptype="general">[한길희/춘천경찰서 경장 : "어떤 분들은 솔직하게 부모님 운전면허 등록해서 탄다. 아니면 면허 있는 다른 지인 친구분이나 이런 걸 빌려서 등록해서 타고 있다."]</p>
<p dmcf-pid="kgeB6qWDda" dmcf-ptype="general">현재 전국 킥보드 운전자의 10% 정도는 여전히 무면허 상태로 운전하는 것으로 경찰은 추정하고 있습니다.</p>
<p dmcf-pid="kwJt83wyHR" dmcf-ptype="general">KBS 뉴스 이청초입니다.</p>
<p dmcf-pid="keRN1fx8HL" dmcf-ptype="general">촬영기자:이장주</p>
<p dmcf-pid="kPVMrWREFM" dmcf-ptype="general">이청초</p>
</section>"""
rx = re.compile(r"<img[^>]*src='([^']+)|r'<p>")
sources = rx.findall(junk)
print(sources)

[]
